In [4]:
import numpy as np
from scipy.special import digamma

In [3]:
class Bayes_UCB_V:
    def __init__(self,x,t,r):
        self.x = x # features
        self.t = t # last listened times of songs
        self.r = r # ratings
        self.lambda_theta_N = 0
        self.eta_theta_N = 0
        self.eta_beta_N = 0
        self.lambda_beta_N = 0
        self.a_N = 0
        self.b_N = 0
        self.D = 0
        self.D_0 = 0
        self.E_0 = 0
        self.mu_theta_0 = 0
        self.mu_beta_0 = 0
        self.a_0 = 0
        self.b_0 = 0
        #where p, K are the dimensionalities of x and t
        self.p = len(x) #?
        self.K = len(t) #?
        
    def variational_inference(self):
        while(!self.is_converged(L())):
            self.update_theta()
            self.update_beta()
            self.update_tau()
        return parameters

    
    # utils
    
    def L(self):
        return self.a_0 * np.log(self.b_0) 
            + (self.a_0 - 1) * (digamma(self.a_N) - np.log(self.b_N))
            - self.b_0 * self.a_N / self.b_N
            - (self.p/2)*np.log(2*np.pi)
            - (1/2)*np.linalg.det(self.D_0)
            + (self.p/2) * (digamma(self.a_N - np.log(self.b_N))
            - (self.a_N/(2*self.b_N)) * (np.trace(self.D_0 * np.linalg.inv(self.lambda_theta_N)) 
                                         + (self.mu_theta_0 - self.expected_theta()).T * np.linalg.inv(self.D_0)* (self.mu_theta_0 - self.expected_theta())
                                        )
            - (self.K/2)*np.log(2*np.pi)
            - (1/2) * np.log(np.linalg.det(self.E_0))
            + (self.K/2) * (digamma(self.a_N - np.log(self.b_N))
            - (self.a_N/(2*self.b_N)) * (np.trace(self.E_0 * np.linalg.inv(self.lambda_beta_N)) 
                                         + (self.mu_beta_0 - self.expected_beta()).T * np.linalg.inv(self.E_0)* (self.mu_beta_0 - self.expected_beta())
                                        )
            - (1/2) * np.log(2*np.pi)
            + (1/2) * (digamma(self.a_N - np.log(self.b_N))
            - (self.a_N/(2*self.b_N)) * self.sum1() 
            + (self.a_N/self.b_N) * self.sum2()
            + (self.K/2) * (1 + np.log(2*np.pi))
            + (1/2) * (np.log(np.linalg.det(np.linalg.inv(self.lambda_beta_N))))
            + (self.p/2) * (1 + np.log(2*np.pi))
            + (1/2) * (np.log(np.linalg.det(np.linalg.inv(self.lambda_theta_N))))
            - (self.a_N-1) * digamma(self.a_N)
            - np.log(self.b_N) + self.a_N           
                       
                       
    def sum1(self):
        pass
    def sum2(self):
        pass
                       
    def is_converged(self):
        #todo fill
        return False
    def update_theta(self):
        pass
    def update_beta(self):
        pass
    def update_tau(self):
        pass
    
    # moments
    
    def expected_beta(self):
        return np.linalg.inv(self.lambda_beta_N)*self.eta_beta_N
    
    def expected_theta(self):
        return np.linalg.inv(self.lambda_theta_N)*self.eta_theta_N
    
    def expected_tau(self):
        return self.a_N/self.b_N
    
    def variance_beta(self):
        return np.linalg.inv(self.lambda_beta_N) + self.expected_beta()*self.expected_beta().T
    
    def variance_theta(self):
        return np.linalg.inv(self.lambda_theta_N) + self.expected_theta()*self.expected_theta().T


SyntaxError: invalid syntax (<ipython-input-3-2826240d5dca>, line 20)